# OSNAP line Lagrangian particle tracking investigation of the cold/fresh blob


### The aim here is an investigation of the source of the cold/fresh blob which appeared in the near-surface Iceland Basin from about 2014. There are a couple of published explanations for the feature: 

- reduced heat/salt transport across RAPID array over several years propagating northwards (Bryden et al. 2020?) or 
- increased contribution from Labrador Sea water flowing into subpolar North Atlantic rather than southwards (Holliday et al. 2020?). Driven I think by specific,  more intermittent, weather near Flemish Cap (though I need to read this again...)

The technique is to track particles back from the OSNAP line, initially for 2 years. Each particle has an associated transport normal to OSNAP, which I use to estimate the contribution to volumes/heat/salt crossing OSNAP northwards from the two main routes - Gulf Stream and Labrador Sea. And see how that changes over the years from 2006 to 2017.

The main conclusion is that from about 2012 Labrador Sea becomes a more important source of surface waters to the Iceland Basin, but there is also a reduction in the temperature and salinity of the water from the Gulf Stream. So probably a balance of both. **But I need to run the particles back further - 2 years is not long enough to pinpoint the pathway for most particles.**

## Comparisons of Viking20 and OSNAP section data

## Technical preamble

In [ ]:
# import matplotlib.colors as colors
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import xarray as xr
from datetime import datetime, timedelta
import seaborn as sns
# from matplotlib.colors import ListedColormap
import cmocean as co
import pandas as pd
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import seawater as sw

from matplotlib import colors as c
from matplotlib import ticker
from xhistogram.xarray import histogram

sns.set(style="darkgrid")

xr.set_options(keep_attrs=True)
np.warnings.filterwarnings('ignore')

sns.set_palette("colorblind")
xr.set_options(keep_attrs=True);


### Set up paths and read in Viking20 section data

In [ ]:
# Project path
project_path = Path.cwd() / '..' / '..' 
project_path = project_path.resolve()

# Section data file
path_data_section = Path('data/processed/sections/') 
fname0 = f'osnapSection2000to09.nc'
fname1 = f'osnapSection2010to18.nc'







In [ ]:
section0 = xr.open_dataset(project_path / path_data_section / fname0)
section1 = xr.open_dataset(project_path / path_data_section / fname1)
section = xr.concat((section0,section1),dim="time_counter")

# add some missing attributes, mostly for automatic axis labels

section.vosaline.attrs['long_name']='salinity'
section.vosaline.attrs['units']='PSU'
section.votemper.attrs['long_name']='potential temperature'
section.votemper.attrs['units']='$\degree C$'
section.c.attrs['long_name']='Distance'
section.c.attrs['units']='m'
section.z.attrs['long_name']='Depth'
section.z.attrs['units']='m'
section.time_counter.attrs["long_name"] = 'Time'

# concat introduces an extra time_counter axis on the variables which didn't have one originally
# so get rid of it

section['ii'] = section['ii'].isel(time_counter=0).drop('time_counter')
section['jj'] = section['jj'].isel(time_counter=0).drop('time_counter')
section['dx'] = section['dx'].isel(time_counter=0).drop('time_counter')
section['lat'] = section['lat'].isel(time_counter=0).drop('time_counter')
section['lon'] = section['lon'].isel(time_counter=0).drop('time_counter')
section['dz'] = section['dz'].isel(time_counter=0).drop('time_counter')
section['mask'] = section['mask'].isel(time_counter=0).drop('time_counter')


In [ ]:
section

### Set up paths and read in OSNAP section data from /Projects/osnap_transports_202008

In [ ]:
# Project path
project_path_osnap = Path('../../..') 
project_path_osnap = project_path_osnap.resolve() / "osnap_transports_202008"


# Parcels track data file
path_data_osnap = Path('data/external/OSNAP') 
filename_streamfunction = 'OSNAP_Streamfunction_201408_201805_2020.nc'
filename_gridded_tsv = 'OSNAP_Gridded_TSV_201408_201805_2020.nc'
filename_moc_timeseries = 'OSNAP_MOC_TimeSeries_201408_201805_2020.nc'

In [ ]:
psi = xr.open_dataset(project_path_osnap / path_data_osnap / filename_streamfunction)
moc = xr.open_dataset(project_path_osnap / path_data_osnap / filename_moc_timeseries)
tsv = xr.open_dataset(project_path_osnap / path_data_osnap / filename_gridded_tsv)

In [ ]:
tsv

##  Hoffmuller diagram of temperature and salinity on the OSNAP section from Viking20x and data

In [ ]:
def plot_hoffmuller():
    fig,ax = plt.subplots(1,2,figsize=(15,13),sharey=True,gridspec_kw={'wspace': 0.1})
    plt.tight_layout()
    section.votemper.isel(z=16).plot(ax=ax[0],vmin=0,vmax=11,cmap=co.cm.thermal,cbar_kwargs={'orientation':'horizontal','pad':0.1})
    section.vosaline.isel(z=16).plot(ax=ax[1],vmin=34.6,vmax=35.5,cmap=co.cm.haline,cbar_kwargs={'orientation':'horizontal','pad':0.1})

    ax[1].set_ylabel('')

    ax[0].ticklabel_format(axis='x', style='sci', scilimits=(-3,3))
    ax[1].ticklabel_format(axis='x', style='sci', scilimits=(-3,3))

    # # Hide x labels and tick labels for top plots and y ticks for right plots.
    # for axis in ax.flat:
    #     axis.label_outer()
    return

In [ ]:
plot_hoffmuller()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(15,5),sharey=True,gridspec_kw={'wspace': 0.1})
tsv.THETA.isel(DEPTH=15).plot(ax=ax[0],vmin=0,cmap=co.cm.thermal,cbar_kwargs={'orientation':'horizontal','pad':0.1})
tsv.PSAL.isel(DEPTH=15).plot(ax=ax[1],vmin=34.6,cmap=co.cm.haline,cbar_kwargs={'orientation':'horizontal','pad':0.1})


## Timeseries of temperature and salinity at points on the OSNAP section. Vertical lines show the earliest and latest particle releases.

In [ ]:
def plot_ts_timeseries():
    fig,ax = plt.subplots(2,figsize=(15,10),sharex=True,gridspec_kw={'wspace': 0.1})
    plt.tight_layout()

    section.votemper.isel(z=15).isel(c=635).plot(ax=ax[0],label='Irminger Basin 37.5$\degree$W')
    section.votemper.isel(z=15).isel(c=742).plot(ax=ax[0],label='Irminger Basin 32.6$\degree$W')
    section.votemper.isel(z=15).isel(c=835).plot(ax=ax[0],label='Iceland Basin 28.0$\degree$W')
    section.votemper.isel(z=15).isel(c=970).plot(ax=ax[0],label='Iceland Basin 22.0$\degree$W')
    section.votemper.isel(z=15).isel(c=1050).plot(ax=ax[0],label='Hatton-Rockall Bank 17.6$\degree$W')
    section.votemper.isel(z=15).isel(c=1200).plot(ax=ax[0],label='Rockall Trough 9.8$\degree$W')
    
    ax[0].axvline(np.datetime64('2006-01-16T12:00:00'))
    ax[0].axvline(np.datetime64('2017-07-16T12:00:00'))

    section.vosaline.isel(z=15).isel(c=635).plot(ax=ax[1],label='Irminger Basin 37.5$\degree$W')
    section.vosaline.isel(z=15).isel(c=742).plot(ax=ax[1],label='Irminger Basin 32.6$\degree$W')
    section.vosaline.isel(z=15).isel(c=835).plot(ax=ax[1],label='Iceland Basin 28.0$\degree$W')
    section.vosaline.isel(z=15).isel(c=970).plot(ax=ax[1],label='Iceland Basin 22.0$\degree$W')
    section.vosaline.isel(z=15).isel(c=1050).plot(ax=ax[1],label='Hatton-Rockall Bank 17.6$\degree$W')
    section.vosaline.isel(z=15).isel(c=1200).plot(ax=ax[1],label='Rockall Trough 9.8$\degree$W')

    ax[1].axvline(np.datetime64('2006-01-16T12:00:00'))
    ax[1].axvline(np.datetime64('2017-07-16T12:00:00'))

    ax[0].set_xlabel('')

    ax[0].legend()
    ax[1].legend()
    
    plt.savefig('plot_ts_times')
    
    return

In [ ]:
plot_ts_timeseries()

In [ ]:
def plot_ts_timeseries_osnap():
    fig,ax = plt.subplots(2,figsize=(15,10),sharex=True,gridspec_kw={'wspace': 0.1})
    plt.tight_layout()

    tsv.THETA.isel(DEPTH=13).isel(LONGITUDE=116).plot(ax=ax[0],label='Irminger Basin 37.5$\degree$W')
    tsv.THETA.isel(DEPTH=13).isel(LONGITUDE=137).plot(ax=ax[0],label='Irminger Basin 32.6$\degree$W')
    tsv.THETA.isel(DEPTH=13).isel(LONGITUDE=170).plot(ax=ax[0],label='Iceland Basin 28.9$\degree$W')
    tsv.THETA.isel(DEPTH=13).isel(LONGITUDE=188).plot(ax=ax[0],label='Iceland Basin 22.0$\degree$W')
    tsv.THETA.isel(DEPTH=13).isel(LONGITUDE=206).plot(ax=ax[0],label='Hatton-Rockall Bank 17.6$\degree$W')
    tsv.THETA.isel(DEPTH=13).isel(LONGITUDE=240).plot(ax=ax[0],label='Rockall Trough 9.8$\degree$W')

    tsv.PSAL.isel(DEPTH=13).isel(LONGITUDE=116).plot(ax=ax[1],label='Irminger Basin 37.5$\degree$W')
    tsv.PSAL.isel(DEPTH=13).isel(LONGITUDE=137).plot(ax=ax[1],label='Irminger Basin 32.6$\degree$W')
    tsv.PSAL.isel(DEPTH=13).isel(LONGITUDE=170).plot(ax=ax[1],label='Iceland Basin 28.9$\degree$W')
    tsv.PSAL.isel(DEPTH=13).isel(LONGITUDE=188).plot(ax=ax[1],label='Iceland Basin 22.0$\degree$W')
    tsv.PSAL.isel(DEPTH=13).isel(LONGITUDE=206).plot(ax=ax[1],label='Hatton-Rockall Bank 17.6$\degree$W')
    tsv.PSAL.isel(DEPTH=13).isel(LONGITUDE=240).plot(ax=ax[1],label='Rockall Trough 9.8$\degree$W')

    ax[0].set_xlabel('')

    ax[0].legend()
    ax[1].legend()
    
    plt.savefig('plot_ts_timeseries.png',bbox_inches='tight')
    

    return

In [ ]:
plot_ts_timeseries_osnap()

### Combined time series

In [ ]:
def plot_ts_timeseries_combined():
    sns.set_palette("deep")
    fig,ax = plt.subplots(2,figsize=(15,10),sharex=True,gridspec_kw={'wspace': 0.1})
    
    sns.set_palette("deep")
    
    section.votemper.isel(z=16).isel(c=1200).plot(ax=ax[0],label='Rockall Trough 9.8$\degree$W',zorder=5)
    section.votemper.isel(z=16).isel(c=1050).plot(ax=ax[0],label='Hatton-Rockall Bank 17.6$\degree$W',zorder=5)
    section.votemper.isel(z=16).isel(c=970).plot(ax=ax[0],label='Iceland Basin 22.0$\degree$W',zorder=5)
    section.votemper.isel(z=16).isel(c=858).plot(ax=ax[0],label='Iceland Basin 28.0$\degree$W',zorder=5)
    section.votemper.isel(z=16).isel(c=742).plot(ax=ax[0],label='Irminger Basin 32.6$\degree$W',zorder=5)
    section.votemper.isel(z=16).isel(c=635).plot(ax=ax[0],label='Irminger Basin 37.5$\degree$W',zorder=5)
    
    ax[0].axvline(np.datetime64('2006-01-16T12:00:00'))
    ax[0].axvline(np.datetime64('2017-07-16T12:00:00'))

    section.vosaline.isel(z=16).isel(c=1200).plot(ax=ax[1],label='Rockall Trough 9.8$\degree$W',zorder=5)
    section.vosaline.isel(z=16).isel(c=1050).plot(ax=ax[1],label='Hatton-Rockall Bank 17.6$\degree$W',zorder=5)
    section.vosaline.isel(z=16).isel(c=970).plot(ax=ax[1],label='Iceland Basin 22.0$\degree$W',zorder=5)
    section.vosaline.isel(z=16).isel(c=858).plot(ax=ax[1],label='Iceland Basin 28.0$\degree$W',zorder=5)
    section.vosaline.isel(z=16).isel(c=742).plot(ax=ax[1],label='Irminger Basin 32.6$\degree$W',zorder=5)
    section.vosaline.isel(z=16).isel(c=635).plot(ax=ax[1],label='Irminger Basin 37.5$\degree$W',zorder=5)
    
#     sns.set_palette("pastel")
    ax[0].set_prop_cycle(None)
    ax[1].set_prop_cycle(None)

    tsv.THETA.isel(DEPTH=15).isel(LONGITUDE=240).plot(ax=ax[0],linewidth=3)
    tsv.THETA.isel(DEPTH=15).isel(LONGITUDE=206).plot(ax=ax[0],linewidth=3)
    tsv.THETA.isel(DEPTH=15).isel(LONGITUDE=188).plot(ax=ax[0],linewidth=3)
    tsv.THETA.isel(DEPTH=15).isel(LONGITUDE=161).plot(ax=ax[0],linewidth=3)
    tsv.THETA.isel(DEPTH=15).isel(LONGITUDE=137).plot(ax=ax[0],linewidth=3)
    tsv.THETA.isel(DEPTH=15).isel(LONGITUDE=116).plot(ax=ax[0],linewidth=3)

    tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=240).plot(ax=ax[1],linewidth=3)
    tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=206).plot(ax=ax[1],linewidth=3)
    tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=188).plot(ax=ax[1],linewidth=3)
    tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=161).plot(ax=ax[1],linewidth=3)
    tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=137).plot(ax=ax[1],linewidth=3)
    tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=116).plot(ax=ax[1],linewidth=3)

    ax[1].axvline(np.datetime64('2006-01-16T12:00:00'))
    ax[1].axvline(np.datetime64('2017-07-16T12:00:00'))
    
    ax[0].set_title('Temperature')
    ax[1].set_title('Salinity')

    ax[0].set_xlabel('')

    ax[0].legend(facecolor='w',framealpha=1)
    ax[1].legend(facecolor='w',framealpha=1)

    plt.tight_layout()
    
    plt.savefig('plot_ts_times',bbox_inches='tight')
    
    return

In [ ]:
plot_ts_timeseries_combined()

In [ ]:
fig,ax = plt.subplots()
tsv.where(tsv.DEPTH < 500).where(tsv.LONGITUDE < -20).where(tsv.LONGITUDE>-30).PSAL.isel(TIME=39).plot(ax=ax,vmin=34.8,vmax=35.5)
ax.set_ylim(500,0)
ax.set_xlim(-30,-20)



In [ ]:
tsv.PSAL.isel(DEPTH=15).isel(LONGITUDE=170)


In [ ]:
tsv.LONGITUDE.data[161]

In [ ]:
section.lon.data[858]